In [75]:
from random import random, choice, randint
from pprint import pprint
from math import sin, cos
from expression_tree import Expression
from binarytree import Node , tree
import numpy as np
import graphviz
import sympy
import copy
# import logging

In [ ]:
# logging.basicConfig(level=logging.DEBUG, file='../log/log1.txt',
#                     format='%(asctime)s  %(levelname)s : %(message)s')

*todo:*
- numpy type refactor
- overwriting variable x of fx with x = roulette
- sympy simplify
- numexpr
- deepcopy
- cut in mse same
- type portability of numpy.types
- jit numba

In [53]:
def operator_map(ch: str, left_sum, right_sum):
    if ch == '+':
        return left_sum + right_sum
    elif ch == '-':
        return left_sum - right_sum
    elif ch == '*':
        return left_sum * right_sum
    elif ch == '/':
        # if right_sum == 0:
        #     return 0
        # TODO : bullshit
        return left_sum / right_sum
    elif ch == '^':
        return left_sum ** right_sum
    elif ch == 'cos':
        return cos(right_sum)
    elif ch == 'sin':
        return sin(right_sum)
    else:
        pass


In [54]:
def evaluate(root: Node, vars: dict):
    """evaluate value of tree from the input for variables using recursive inorder traverse"""
    if root is None: # empty tree
        return 0
    if root.left is None and root.right is None:  # leaf node
        if root.value in vars:
            return vars[root.value]
            # TODO : add check for is digit & in vars throw
        else:
            return int(root.value)
            # TODO : just int not float sin(1.45)
            # TODO : sin is radian
    left_sum = evaluate(root.left, vars)
    right_sum = evaluate(root.right, vars)
    return operator_map(root.value, left_sum, right_sum)

In [55]:
def MSE(actual, predictions):
    """Mean Squared Error"""
    return np.square(np.subtract(actual, predictions)).mean()

In [56]:
# strscope = "abcde"
# scope = [c for c in strscope]
# num_exp=randint(1,5),
# randexpr = generate_expressions(scope,num_exp=int(input("enter num_exp: ")),num_ops=int(input("enter num_ops: ")))
# for index,expression in enumerate(randexpr):
#     print(expression)
#     # print(f'{index=}------>',tree)
# test = Expression(expression = "x+sin(90)^2*y",
#                     operators = {'+', 'sin', '^', '*'},
#                     operators_info = {'+': (2, 1), '*': (2, 2),'^': (2, 3), 'sin': (1, 4)},
#                     operators_associativity = {'+': 'LR', '*': 'LR','^': 'RL', 'sin': 'RL'},
#                     variables = {'x', 'y'})
expr = "x+sin(90)^2*y"
expr = '1+2*x^9/cos(x^2)+2*y^9'
expr = '1+2*x^9/cos(x^2)^2+2*y^9'
expr = "x+2*y^2"
op = {'+', '-', '*', '/', '^',
      'sin', 'cos'}
op_info = {'+': (2, 1), '-': (2, 1),
           '*': (2, 2), '/': (2, 2),
           '^': (2, 3),
           'sin': (1, 4), 'cos': (1, 4)}
assotiation = {'+': 'LR', '-': 'LR',
               '*': 'LR', '/': 'LR',
               '^': 'RL',
               'sin': 'RL', 'cos': 'RL'}
varchar = {'x', 'y'}
test = Expression(expression=expr, operators=op, operators_info=op_info,
                  operators_associativity=assotiation, variables=varchar)
print(expr)
print(test.tree())
test.tree().graphviz()
# pprint(test._tokens)
# pprint(list(expr))
print(evaluate(test.tree(), {'x': 4, 'y': 2}))

x+2*y^2

  +__
 /   \
x     *__
     /   \
    2     ^
         / \
        y   2

12


In [57]:
# test.tree().graphviz()

In [58]:
UNARIES = ["sqrt(%s)", "exp(%s)", "log(%s)", "sin(%s)", "cos(%s)", "tan(%s)",
           "sinh(%s)", "cosh(%s)", "tanh(%s)", "asin(%s)", "acos(%s)",
           "atan(%s)", "-%s"]
BINARIES = ["%s + %s", "%s - %s", "%s * %s", "%s / %s", "%s ^ %s"]
BINARIES = ["%s + %s", "%s - %s", "%s * %s"]


PROP_PARANTHESIS = 0.4
PROP_BINARY = 1.0

In [59]:
def generate_expressions(scope, num_exp, num_ops):
    print(f"{num_ops=}")
    print(f"{num_exp=}")

    scope = list(scope)  # make a copy first, append as we go
    for _ in range(num_ops):
        if random() < PROP_BINARY:  # decide unary or binary operator
            ex = choice(BINARIES) % (choice(scope), choice(scope))
            if random() < PROP_PARANTHESIS:
                ex = "(%s)" % ex
            scope.append(ex)
        else:
            scope.append(choice(UNARIES) % choice(scope))
    return scope[-num_exp:]  # return most recent expressions

In [61]:
def main():
    # test()
    # np.set_printoptions(precision=3)

    x = np.array(range(1, 10), dtype='float')
    y = f(x)
    print(f"{x=}")
    print(f"{y=}")
    print(f"{scope=}")
    randexpr_arr = generate_expressions(scope, num_exp=5, num_ops=5)
    randexpr_exp = []
    randexpr_tree = []
    y_pred = []

    for index, expression in enumerate(randexpr_arr):
        test = Expression(expression=expression, operators=op, operators_info=op_info,
                          operators_associativity=assotiation, variables=varchar)
        randexpr_exp.append(test)
        randexpr_tree.append(test.tree())
        y_pred.append(evaluate_vectorized(randexpr_tree[index], x))
        print(f"{index}:", randexpr_exp[index], "& ", expression)
        pprint(y_pred[index])
        print(randexpr_tree[index])

    mse = np.array([MSE(y, gg) for gg in y_pred])
    print(mse)

    genetic_algo(population=randexpr_tree, mutation_probability=0)

In [62]:
def f(x):
    return x**x + x

In [63]:
op = {'+', '-', '*', '/', '^',
      'sin', 'cos'}
op_info = {'+': (2, 1), '-': (2, 1),
           '*': (2, 2), '/': (2, 2),
           '^': (2, 3),
           'sin': (1, 4), 'cos': (1, 4)}
assotiation = {'+': 'LR', '-': 'LR',
               '*': 'LR', '/': 'LR',
               '^': 'RL',
               'sin': 'RL', 'cos': 'RL'}
varchar = {'x'}
strscope = 'x'
scope = [c for c in strscope] 
# todo : expand scope


In [64]:
def evaluate_vectorized(root: Node, vars: np.array):
    if root is None: # empty tree
        return 0
    if root.left is None and root.right is None: # leaf node
        return vars
        # if root.value in scope:
        #     return vars[]
        #     # TODO : add check for is digit & in vars throw
        # else:
        #     return int(root.value)
        #     # TODO : just int not float sin(1.45)
        #     # TODO : sin is radian
    left_sum = evaluate_vectorized(root.left, vars)
    right_sum = evaluate_vectorized(root.right, vars)
    return operator_map(root.value, left_sum, right_sum)

In [65]:
x = np.array(range(1, 8), dtype=np.longdouble)
# todo range from zero but / has problem
y = f(x)
print(f"{x=}")
print(f"{y=}")
print(f"{scope=}")

x=array([1., 2., 3., 4., 5., 6., 7.], dtype=float64)
y=array([2.0000e+00, 6.0000e+00, 3.0000e+01, 2.6000e+02, 3.1300e+03,
       4.6662e+04, 8.2355e+05], dtype=float64)
scope=['x']


In [66]:
randexpr_arr = generate_expressions(scope, num_exp=10, num_ops=10)
randexpr_arr

num_ops=10
num_exp=10


['x + x',
 '(x + x + x)',
 'x + x - x + x',
 '(x + x - x + x - x + x - x + x)',
 '(x + x + x) - (x + x + x)',
 '(x + x - x + x * (x + x - x + x - x + x - x + x))',
 '(x + x - x + x - x + x - x + x) * (x + x - x + x - x + x - x + x)',
 'x * (x + x + x) - (x + x + x)',
 '(x + x - x + x - x + x - x + x) * (x + x - x + x - x + x - x + x) - x + x',
 '((x + x - x + x - x + x - x + x) + (x + x - x + x - x + x - x + x) * (x + x - x + x - x + x - x + x) - x + x)']

In [67]:
simpled = [str(sympy.sympify(str(x))) for x in randexpr_arr]
# simpled = list({str(sympy.sympify(str(x))) for x in generate_expressions(scope, num_exp=10, num_ops=10)})
pprint(simpled)

['2*x',
 '3*x',
 '2*x',
 '2*x',
 '0',
 '2*x**2 + x',
 '4*x**2',
 '3*x**2 - 3*x',
 '4*x**2',
 '4*x**2 + 2*x']


In [68]:
randexpr_exp = []
randexpr_tree = []
y_pred = []
for index, expression in enumerate(randexpr_arr):
    test = Expression(expression=expression, operators=op, operators_info=op_info,
                      operators_associativity=assotiation, variables=varchar)
    randexpr_exp.append(test)
    randexpr_tree.append(test.tree())
    y_pred.append(evaluate_vectorized(randexpr_tree[index], x))
    print(f"{index}:", randexpr_exp[index], "& ", expression)
    pprint(y_pred[index])
    print(randexpr_tree[index])
mse = np.array([MSE(y, gg) for gg in y_pred], dtype = np.longdouble)
print(mse)


0: x+x &  x + x
array([ 2.,  4.,  6.,  8., 10., 12., 14.], dtype=float64)

  +
 / \
x   x

1: (x+x+x) &  (x + x + x)
array([ 3.,  6.,  9., 12., 15., 18., 21.], dtype=float64)

  +__
 /   \
x     +
     / \
    x   x

2: x+x-x+x &  x + x - x + x
array([0., 0., 0., 0., 0., 0., 0.], dtype=float64)

  +__
 /   \
x     -__
     /   \
    x     +
         / \
        x   x

3: (x+x-x+x-x+x-x+x) &  (x + x - x + x - x + x - x + x)
array([0., 0., 0., 0., 0., 0., 0.], dtype=float64)

  +__
 /   \
x     -__
     /   \
    x     +__
         /   \
        x     -__
             /   \
            x     +__
                 /   \
                x     -__
                     /   \
                    x     +
                         / \
                        x   x

4: (x+x+x)-(x+x+x) &  (x + x + x) - (x + x + x)
array([0., 0., 0., 0., 0., 0., 0.], dtype=float64)

    ______-__
   /         \
  +__         +__
 /   \       /   \
x     +     x     +
     / \         / \
    x   x       x   x

5: (x

In [69]:
# genetic_algo(population=randexpr_tree, mse=mse, mutation_probability=0)

In [70]:
def roulette_wheel_selection(population, fit_vec: np.array):
    """Roulette Wheel Selection for minimizing problem"""
    maxfit = np.max(fit_vec)
    fitvec2 = maxfit - fit_vec
    probmax = np.sum(fitvec2)
    selection_probs = fitvec2 / probmax
    return population[np.random.choice(len(population), p=selection_probs)]
    # TODO : negetive mse probability

In [72]:
first_tree = roulette_wheel_selection(randexpr_tree, mse)
second_tree = roulette_wheel_selection(randexpr_tree, mse)
print(first_tree)
print(second_tree)


[0.1003966  0.12549431 0.05019946 0.05019946 0.05019946 0.07529832
 0.05019946 0.49801292 0.         0.        ]
[9.71996520e+10 9.71979202e+10 9.72031157e+10 9.72031157e+10
 9.72031157e+10 9.72013839e+10 9.72031157e+10 9.71722157e+10
 9.72065796e+10 9.72065796e+10]
[0.1003966  0.12549431 0.05019946 0.05019946 0.05019946 0.07529832
 0.05019946 0.49801292 0.         0.        ]
[9.71996520e+10 9.71979202e+10 9.72031157e+10 9.72031157e+10
 9.72031157e+10 9.72013839e+10 9.72031157e+10 9.71722157e+10
 9.72065796e+10 9.72065796e+10]

    __________-__
   /             \
  *__             +__
 /   \           /   \
x     +__       x     +
     /   \           / \
    x     +         x   x
         / \
        x   x


  +__
 /   \
x     -__
     /   \
    x     +__
         /   \
        x     -__
             /   \
            x     +__
                 /   \
                x     -__
                     /   \
                    x     +
                         / \
                        

In [79]:
def swapNode(x_node,y_node):
    x_node.__dict__, y_node.__dict__ = y_node.__dict__, x_node.__dict__

In [77]:
def crossover(x: Node, y:Node):
    print("______________________________________________________")
    print(x)
    print(y)
    print("++++++++++")
    x_copy = copy.deepcopy(x)
    y_copy = copy.deepcopy(y)
    inorder_x = x_copy.inorder
    inorder_y = y_copy.inorder
    # print(inorder_x, '\n',inorder_y)
    x_node = Node(1)
    y_node = Node(1)
    while x_node.value not in op:
        x_node = choice(inorder_x)
    while y_node.value not in op:
        y_node = choice(inorder_y)
    print(x_node)
    print(y_node)
    swapNode(x_node, y_node)
    print(x_copy)
    print(y_copy)
    return x_copy, y_copy


child = crossover(first_tree, second_tree)


______________________________________________________

    __________-__
   /             \
  *__             +__
 /   \           /   \
x     +__       x     +
     /   \           / \
    x     +         x   x
         / \
        x   x


  +__
 /   \
x     -__
     /   \
    x     +__
         /   \
        x     -__
             /   \
            x     +__
                 /   \
                x     -__
                     /   \
                    x     +
                         / \
                        x   x

++++++++++

  +__
 /   \
x     +
     / \
    x   x


  -__
 /   \
x     +
     / \
    x   x


    __________-__
   /             \
  *__             -__
 /   \           /   \
x     +__       x     +
     /   \           / \
    x     +         x   x
         / \
        x   x


  +__
 /   \
x     -__
     /   \
    x     +__
         /   \
        x     -__
             /   \
            x     +__
                 /   \
                x     +__
                   

In [82]:
def genetic_algo(population, mse, mutation_probability, saving:float):
    
    for abc in range(10):
        new_population = []
        len_population = len(population)
        count_saving = int(saving* len(population))
        y_pred = [evaluate_vectorized(tree,x) for tree in population]
        mse = [MSE(y, gg) for gg in y_pred]
        population = [guy for _,guy in sorted(zip(mse,population),reverse=True)]
        mse = sorted(mse,reverse=True)
        new_population += population[:count_saving]
        for i in range(len_population/2):
            x = roulette_wheel_selection(population, mse)
            y = roulette_wheel_selection(population, mse)
            child1,child2 = crossover(x, y)
            # if (random() < mutation_probability):
            #     mutate(child)
            new_population.append(child1)
            new_population.append(child2)
        population = new_population
        